# LLaMA-Factory webui in Singularity container

## 初始環境設定
- 重新reload kernel, 並切換到 kernel Image_S-work-LlamaFactory_c00cjz00_cuda11.8_pytorch_2.1.2-cuda11.8-cudnn8-devel

In [1]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [2]:
# GPU確認
!nvidia-smi
import torch
torch.cuda.is_available()

Tue Mar 12 08:11:31 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3E:00.0 Off |                    0 |
| N/A   30C    P0    56W / 300W |  13896MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:B1:00.0 Off |                    0 |
| N/A   

True

In [4]:
# 套件安裝 (此映像檔已經安裝以下套件, 若使用其他映像檔, 請移除 ## 安裝以下套件)
##!pip install llmtuner==0.5.3 deepspeed==0.13.1  bitsandbytes==0.42.0 -q 
##!pip install deepspeed==0.13.1 -q
##!pip install bitsandbytes==0.42.0 -q

## SFT (sft in mutiple GPU)

In [3]:
%%bash
## qlora in single GPU (bitsandbytes) 
## 啟動 Llama-Factory Cmd
## 請記得修改最後一行, 給予huggingface token,  HF_TOKEN='hf_' ?
## 記錄檔案為 llamafactory.log 
## 量化請加上     --quantization_bit 4 \
## --model_name_or_path /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf \
## meta-llama/Llama-2-7b-hf

cd LLaMA-Factory

HF_TOKEN='hf_' deepspeed --num_gpus 2 src/train_bash.py \
    --deepspeed examples/full_multi_gpu/ds_z3_config.json \
    --stage sft \
    --do_train \
    --model_name_or_path /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf \
    --dataset alpaca_gpt4_en,glaive_toolcall \
    --dataset_dir data \
    --template default \
    --finetuning_type lora \
    --lora_target q_proj,v_proj \
    --output_dir ../saves/LLaMA2-7B/lora/sft \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 1024 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --evaluation_strategy steps \
    --load_best_model_at_end \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --val_size 0.1 \
    --plot_loss \
    --fp16 

[2024-03-12 08:11:38,311] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-03-12 08:11:39,771] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-03-12 08:11:39,772] [INFO] [runner.py:568:main] cmd = /opt/conda/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMV19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None src/train_bash.py --deepspeed examples/full_multi_gpu/ds_z3_config.json --stage sft --do_train --model_name_or_path /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf --dataset alpaca_gpt4_en,glaive_toolcall --dataset_dir data --template default --finetuning_type lora --lora_target q_proj,v_proj --output_dir ../saves/LLaMA2-7B/lora/sft --overwrite_cache --overwrite_output_dir --cutoff_len 1024 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --gradient_accumulation_steps 8 --lr_s

[INFO|tokenization_utils_base.py:2044] 2024-03-12 08:11:49,636 >> loading file tokenizer.model
[INFO|tokenization_utils_base.py:2044] 2024-03-12 08:11:49,636 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2044] 2024-03-12 08:11:49,636 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2044] 2024-03-12 08:11:49,636 >> loading file tokenizer_config.json
[INFO|tokenization_utils_base.py:2044] 2024-03-12 08:11:49,636 >> loading file tokenizer.json


03/12/2024 08:11:49 - INFO - llmtuner.data.template - Add pad token: </s>
03/12/2024 08:11:49 - INFO - llmtuner.data.loader - Loading dataset alpaca_gpt4_data_en.json...
03/12/2024 08:11:49 - INFO - llmtuner.data.template - Add pad token: </s>


Converting format of dataset: 100%|██████████| 1000/1000 [00:00<00:00, 27596.83 examples/s]


03/12/2024 08:11:50 - INFO - llmtuner.data.loader - Loading dataset glaive_toolcall_10k.json...
03/12/2024 08:11:51 - WARNING - llmtuner.data.utils - Checksum failed: mismatched SHA-1 hash value at data/glaive_toolcall_10k.json.


...

[2024-03-12 08:12:29,013] [INFO] [launch.py:315:sigkill_handler] Killing subprocess 13585


  0%|          | 0/336 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/subprocess.py", line 1209, in wait


[2024-03-12 08:12:29,224] [INFO] [launch.py:315:sigkill_handler] Killing subprocess 13585


    return self._wait(timeout=timeout)
  File "/opt/conda/lib/python3.10/subprocess.py", line 1959, in _wait
    (pid, sts) = self._try_wait(0)
  File "/opt/conda/lib/python3.10/subprocess.py", line 1917, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/bin/deepspeed", line 6, in <module>
    main()
  File "/opt/conda/lib/python3.10/site-packages/deepspeed/launcher/runner.py", line 584, in main
    result.wait()
  File "/opt/conda/lib/python3.10/subprocess.py", line 1222, in wait
    self._wait(timeout=sigint_timeout)
  File "/opt/conda/lib/python3.10/subprocess.py", line 1953, in _wait
    time.sleep(delay)
KeyboardInterrupt

KeyboardInterrupt



[2024-03-12 08:12:29,264] [INFO] [launch.py:315:sigkill_handler] Killing subprocess 13586
[2024-03-12 08:12:29,415] [INFO] [launch.py:315:sigkill_handler] Killing subprocess 13585
[2024-03-12 08:12:29,415] [INFO] [launch.py:315:sigkill_handler] Killing subprocess 13586
[2024-03-12 08:12:29,521] [INFO] [launch.py:324:sigkill_handler] Main process received SIGINT, exiting
